In [2]:
# Data Collection
# Data Precprocessing

# EDA
# Conclusion

In [39]:
! pip install beautifulsoup4

In [40]:
! pip install plotly

In [41]:
from bs4 import BeautifulSoup as soup
from datetime import date, datetime
from urllib.request import Request, urlopen

import pandas as pd
import numpy as np

In [42]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import seaborn as sns
import gc
import warnings
warnings.filterwarnings("ignore")

 


# Web scraping

In [43]:
today = datetime.now()
yesterday_str = "%s %d,%d" %(date.today().strftime("%b"), today.day-1, today.year)
yesterday_str

'Jan 27,2022'

In [44]:
url = "https://www.worldometers.info/coronavirus/#countries"
req = Request(url , headers={'User-Agent':"Mozilla/5.0"})

webpage = urlopen(req)
print(webpage)
page_soup = soup(webpage, "html.parser")

In [45]:
table = page_soup.findAll("table",{"id":"main_table_countries_yesterday"})

containers = table[0].findAll("tr",{"style":""})
title = containers[0]
del containers[0]

all_data = []
clean = True
for country in containers:
    country_data = []
    country_container = country.findAll("td")
    
    if country_container[1].text == "China":
        continue
    for i in range(1, len(country_container)):
        final_feature = country_container[i].text
        if clean :
            if i !=1 and i != len(country_container)-1:
                final_feature = final_feature.replace(",","")
                
                if final_feature.find('+') != -1:
                    final_feature = final_feature.replace("+","")
                    final_feature = float(final_feature)
                    
                elif final_feature.find('-') != -1:
                    final_feature = final_feature.replace("-","")
                    final_feature = float(final_feature)*-1
                    
        if final_feature == 'N/A':
            final_feature = 0
        elif final_feature == "" or final_feature == " ":
            final_feature = -1
            
        country_data.append(final_feature)
    all_data.append(country_data)

In [46]:
all_data

[['World',
  '366709951',
  3623232.0,
  '5656234',
  10261.0,
  '290064796',
  2578322.0,
  '70988921',
  '95948',
  '47045',
  '725.6',
  -1,
  -1,
  -1,
  'All',
  '\n',
  -1,
  -1,
  -1,
  -1,
  -1],
 ['USA',
  '74695333',
  497351.0,
  '902140 ',
  2689.0,
  '45614212',
  192894.0,
  '28178981',
  '25189',
  '223604',
  '2701',
  '894319678',
  '2677193',
  '334051237 ',
  'North America',
  '4',
  '370',
  '0',
  '1489',
  '8',
  '84,355'],
 ['India',
  '40622709',
  251209.0,
  '492356 ',
  627.0,
  '38024771',
  347443.0,
  '2105582',
  '8944',
  '28989',
  '351',
  '722166248',
  '515351',
  '1401309721 ',
  'Asia',
  '34',
  '2846',
  '2',
  '179',
  '0.4',
  '1,503'],
 ['Brazil',
  '24782922',
  228972.0,
  '625169 ',
  662.0,
  '22098157',
  61989.0,
  '2059596',
  '8318',
  '115306',
  '2909',
  '63776166',
  '296727',
  '214932449 ',
  'South America',
  '9',
  '344',
  '3',
  '1065',
  '3',
  '9,583'],
 ['France',
  '18122724',
  392168.0,
  '130015 ',
  268.0,
  '113817

In [49]:
df = pd.DataFrame(all_data)
df.drop([15,16,17,18,19,20], inplace=True, axis=1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,World,366709951,3623232.0,5656234,10261.0,290064796,2578322.0,70988921,95948,47045,725.6,-1,-1,-1,All
1,USA,74695333,497351.0,902140,2689.0,45614212,192894.0,28178981,25189,223604,2701,894319678,2677193,334051237,North America
2,India,40622709,251209.0,492356,627.0,38024771,347443.0,2105582,8944,28989,351,722166248,515351,1401309721,Asia
3,Brazil,24782922,228972.0,625169,662.0,22098157,61989.0,2059596,8318,115306,2909,63776166,296727,214932449,South America
4,France,18122724,392168.0,130015,268.0,11381705,327956.0,6611004,3760,276681,1985,223502325,3412229,65500398,Europe


In [50]:
column_labels = ["Country", "Total Cases", "New Cases", "Total Deaths", "New Deaths", "Total Recovered", "New Recovered", "Active Cases", "Serious/Critical","Total cases/1M","Deaths/1M","Total Tests","Test/1M","Population","Continent"]
df.columns = column_labels
df.head()


,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent
0,World,366709951,3623232.0,5656234,10261.0,290064796,2578322.0,70988921,95948,47045,725.6,-1,-1,-1,All
1,USA,74695333,497351.0,902140,2689.0,45614212,192894.0,28178981,25189,223604,2701,894319678,2677193,334051237,North America
2,India,40622709,251209.0,492356,627.0,38024771,347443.0,2105582,8944,28989,351,722166248,515351,1401309721,Asia
3,Brazil,24782922,228972.0,625169,662.0,22098157,61989.0,2059596,8318,115306,2909,63776166,296727,214932449,South America
4,France,18122724,392168.0,130015,268.0,11381705,327956.0,6611004,3760,276681,1985,223502325,3412229,65500398,Europe


In [51]:
for label in df.columns:
    if label != "Country" and label != "Continent":
        df[label] = pd.to_numeric(df[label])

In [52]:
df["%Inc Cases"] = df["New Cases"] / df["Total Cases"]*100
df["%Inc Deaths"] = df["New Deaths"] / df["Total Deaths"]*100
df["%Inc Recovered"] = df["New Recovered"] / df["Total Recovered"]*100

In [53]:
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent,%Inc Cases,%Inc Deaths,%Inc Recovered
0,World,366709951,3623232.0,5656234,10261.0,290064796,2578322.0,70988921,95948,47045.0,725.6,-1,-1,-1,All,0.988038,0.181410,0.888878
1,USA,74695333,497351.0,902140,2689.0,45614212,192894.0,28178981,25189,223604.0,2701.0,894319678,2677193,334051237,North America,0.665839,0.298069,0.422881
2,India,40622709,251209.0,492356,627.0,38024771,347443.0,2105582,8944,28989.0,351.0,722166248,515351,1401309721,Asia,0.618395,0.127347,0.913728
3,Brazil,24782922,228972.0,625169,662.0,22098157,61989.0,2059596,8318,115306.0,2909.0,63776166,296727,214932449,South America,0.923910,0.105891,0.280517
4,France,18122724,392168.0,130015,268.0,11381705,327956.0,6611004,3760,276681.0,1985.0,223502325,3412229,65500398,Europe,2.163957,0.206130,2.881431


# EDA

In [59]:
cases = df[["Total Recovered","Active Cases","Total Deaths"]].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ["Type","Total"]

cases_df["Percentage"] = np.round(100*cases_df["Total"]/np.sum(cases_df["Total"]),2)
cases_df["Virus"] = ["Covid-19" for i in range(len(cases_df))]
cases_df

,Type,Total,Percentage,Virus
0,Total Recovered,290064796,79.10,Covid-19
1,Active Cases,70988921,19.36,Covid-19
2,Total Deaths,5656234,1.54,Covid-19


In [61]:
fig = px.bar(cases_df, x="Virus", y="Percentage", color="Type",hover_data=["Total"])
fig.show()

In [56]:
cases = df[["New Recovered","New Cases","New Deaths"]].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ["Type","Total"]

cases_df["Percentage"] = np.round(100*cases_df["Total"]/np.sum(cases_df["Total"]),2)
cases_df["Virus"] = ["Covid-19" for i in range(len(cases_df))]
cases_df

,Type,Total,Percentage,Virus
0,New Recovered,2578322.0,41.51,Covid-19
1,New Cases,3623232.0,58.33,Covid-19
2,New Deaths,10261.0,0.17,Covid-19


In [58]:
fig = px.bar(cases_df, x="Virus", y="Percentage", color="Type",hover_data=["Total"])
fig.show()

In [25]:
per = np.round(df[["%Inc Cases","%Inc Deaths","%Inc Recovered"]].loc[0],2)
per_df = pd.DataFrame(per)

per_df.columns = ["Percentage"]

fig = go.Figure()
fig.add_trace(go.Bar(x=per_df.index, y=per_df["Percentage"], marker_color=["Yellow","blue","red"]))



# Continent

In [62]:
continent_df = df.groupby("Continent").sum().drop("All")
continent_df = continent_df.reset_index()
continent_df

,Continent,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Population,%Inc Cases,%Inc Deaths,%Inc Recovered
0,Africa,10920302,38057.0,238321,358.0,9696147,41365.0,946206,2625,1501412.0,16727.0,92239678,9381899,1390938593,7.636686,-103.307219,17.027657
1,Asia,97637248,684547.0,1280377,1384.0,90178717,870219.0,6178154,26996,3083315.0,26855.0,1645768483,70784294,3222638985,37.709468,-31.204516,33.222821
2,Australia/Oceania,2612205,86305.0,5770,66.0,2102628,85908.0,456395,424,500989.0,4943.0,67418182,7421823,42724700,66.886504,345.103095,72.211878
3,Europe,120650241,1791902.0,1606624,3239.0,90604749,1058305.0,28438867,19060,9676163.0,98594.0,1903500285,162434867,748339316,66.888470,108.362451,39.293831
4,North America,87406015,595480.0,1310234,3490.0,56209346,295302.0,29877245,32040,5140129.0,45027.0,1001666389,76377753,596358628,36.413495,-25.814463,31.689102
5,South America,47377311,426847.0,1210250,1643.0,38964711,226978.0,3629267,14740,1447817.0,29018.0,199422208,11275518,436341327,9.450407,104.322733,9.718395


In [63]:
def continent_visualization(v_list):
    for label in v_list:
        c_df = continent_df[["Continent",label]]
        c_df["Percentage"]= np.round(100*c_df[label]/np.sum(c_df[label]),2)
        c_df['Virus'] = ["Covid-19" for i in range(len(c_df))]
        
        fig = px.bar(c_df, x="Virus", y="Percentage", color="Continent",hover_data=[label])
        fig.update_layout(title={"text":label})
        fig.show()

In [64]:
cases_list = ["Total Cases","Active Cases","New Cases","Serious/Critical","Total cases/1M"]

deaths_list = ["Total Deaths","New Deaths","Deaths/1M"]

recovered_list = ["Total Recovered","New Recovered","%Inc Recovered"]

In [30]:
continent_visualization(cases_list)

# Countries

In [65]:
df = df.drop([len(df)-1])
country_df = df.drop([0])

country_df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent,%Inc Cases,%Inc Deaths,%Inc Recovered
1,USA,74695333,497351.0,902140,2689.0,45614212,192894.0,28178981,25189,223604.0,2701.0,894319678,2677193,334051237,North America,0.665839,0.298069,0.422881
2,India,40622709,251209.0,492356,627.0,38024771,347443.0,2105582,8944,28989.0,351.0,722166248,515351,1401309721,Asia,0.618395,0.127347,0.913728
3,Brazil,24782922,228972.0,625169,662.0,22098157,61989.0,2059596,8318,115306.0,2909.0,63776166,296727,214932449,South America,0.923910,0.105891,0.280517
4,France,18122724,392168.0,130015,268.0,11381705,327956.0,6611004,3760,276681.0,1985.0,223502325,3412229,65500398,Europe,2.163957,0.206130,2.881431
5,UK,16244804,96647.0,155040,338.0,12961845,180533.0,3127919,561,237337.0,2265.0,448159099,6547618,68446125,Europe,0.594941,0.218008,1.392803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Montserrat,161,-1.0,1,-1.0,148,-1.0,12,2,32219.0,200.0,7692,1539324,4997,North America,-0.621118,-100.000000,-0.675676
213,Falkland Islands,85,-1.0,-1,-1.0,0,0.0,0,-1,23358.0,-1.0,8528,2343501,3639,South America,-1.176471,100.000000,NaN
214,Vatican City,29,-1.0,-1,-1.0,27,-1.0,2,-1,36070.0,-1.0,-1,-1,804,Europe,-3.448276,100.000000,-3.703704
215,Samoa,27,-1.0,-1,-1.0,3,-1.0,24,-1,135.0,-1.0,-1,-1,200485,Australia/Oceania,-3.703704,100.000000,-33.333333


In [66]:
look =5
country = country_df.columns[1:14]

fig = go.Figure()
c=0
for i in country_df.index:
    if c < look:
        fig.add_trace(go.Bar(name=country_df['Country'][i], x=country, y=country_df.loc[i][1:14]))
    else:
        break
    c +=1
                      
fig.update_layout(title = {"text":f'{look}'})
fig.show()
                      
                      
                      
                      
                      

In [36]:
look =5
country = country_df.columns[1:14]

fig = go.Figure()
c=0
for i in country_df.index:
    if c < look:
        fig.add_trace(go.Bar(name=country_df['Country'][i], x=country, y=country_df.loc[i][1:14]))
    else:
        break
    c +=1
                      
fig.update_layout(title = {"text":f'top{look} countries affected'}, yaxis_type="log")
fig.show()
                      
                      